# Querying Data in _pymongo_

In this session we will expand upon the work in the previous ("3-02: Connecting to a MongoDB Database") by covering different types of queries and data manipulations. As such, we will begin by re-establishing our connection to the sample database created previously:

In [6]:
from pymongo import MongoClient
import urllib

# final version
con_str = "mongodb+srv://michaelm:" + urllib.parse.quote_plus("B!gD@t@T3(h") + "@cluster0.m2kzp.mongodb.net/sample_analytics?retryWrites=true&w=majority"

client = MongoClient(con_str)

db = client.sample_analytics # database
collection = db.customers # select the customers collection

In the last session we randomly selected a record from the collection. Here we'll re-run this to demonstrate after other queries we may run:

In [7]:
record = collection.find_one()
record

OperationFailure: bad auth : Authentication failed., full error: {'ok': 0, 'errmsg': 'bad auth : Authentication failed.', 'code': 8000, 'codeName': 'AtlasError'}

Firstly we may note that the database is returning a dictionary. We can use this to extract the elements we want:

In [3]:
print(record['_id'])
print(record['accounts'][0:3])

5ca4bbcea2dd94ee58162a68
[371138, 324287, 276528]


We can use the "ObjectId" above to demonstrate querying by ID:

In [4]:
collection.find_one({"_id": record['_id']})

{'_id': ObjectId('5ca4bbcea2dd94ee58162a68'),
 'username': 'fmiller',
 'name': 'Elizabeth Ray',
 'address': '9286 Bethany Glens\nVasqueztown, CO 22939',
 'birthdate': datetime.datetime(1977, 3, 2, 2, 20, 31),
 'email': 'arroyocolton@gmail.com',
 'active': True,
 'accounts': [371138, 324287, 276528, 332179, 422649, 387979],
 'tier_and_details': {'0df078f33aa74a2e9696e0520c1a828a': {'tier': 'Bronze',
   'id': '0df078f33aa74a2e9696e0520c1a828a',
   'active': True,
   'benefits': ['sports tickets']},
  '699456451cc24f028d2aa99d7534c219': {'tier': 'Bronze',
   'benefits': ['24 hour dedicated line', 'concierge services'],
   'active': True,
   'id': '699456451cc24f028d2aa99d7534c219'}}}

Alternatively we can use _find()_ to search the whole collection:

In [5]:
count = 0

for doc in collection.find():
    if count < 5:
        print(doc)
        count += 1
    else:
        break

{'_id': ObjectId('5ca4bbcea2dd94ee58162a68'), 'username': 'fmiller', 'name': 'Elizabeth Ray', 'address': '9286 Bethany Glens\nVasqueztown, CO 22939', 'birthdate': datetime.datetime(1977, 3, 2, 2, 20, 31), 'email': 'arroyocolton@gmail.com', 'active': True, 'accounts': [371138, 324287, 276528, 332179, 422649, 387979], 'tier_and_details': {'0df078f33aa74a2e9696e0520c1a828a': {'tier': 'Bronze', 'id': '0df078f33aa74a2e9696e0520c1a828a', 'active': True, 'benefits': ['sports tickets']}, '699456451cc24f028d2aa99d7534c219': {'tier': 'Bronze', 'benefits': ['24 hour dedicated line', 'concierge services'], 'active': True, 'id': '699456451cc24f028d2aa99d7534c219'}}}
{'_id': ObjectId('5ca4bbcea2dd94ee58162a69'), 'username': 'valenciajennifer', 'name': 'Lindsay Cowan', 'address': 'Unit 1047 Box 4089\nDPO AA 57348', 'birthdate': datetime.datetime(1994, 2, 19, 23, 46, 27), 'email': 'cooperalexis@hotmail.com', 'accounts': [116508], 'tier_and_details': {'c06d340a4bad42c59e3b6665571d2907': {'tier': 'Plati

We can also use _find()_ to return all results that meet some criteria, for instance:

In [6]:
import datetime

count = 0

# midnight on the 13th June 1981
date_cutoff = datetime.datetime(1981, 6, 13, 0, 0, 0)

for doc in collection.find({"birthdate": {"$lt": date_cutoff}}):
    if count < 5:
        print(doc)
        count += 1
    else:
        break

{'_id': ObjectId('5ca4bbcea2dd94ee58162a68'), 'username': 'fmiller', 'name': 'Elizabeth Ray', 'address': '9286 Bethany Glens\nVasqueztown, CO 22939', 'birthdate': datetime.datetime(1977, 3, 2, 2, 20, 31), 'email': 'arroyocolton@gmail.com', 'active': True, 'accounts': [371138, 324287, 276528, 332179, 422649, 387979], 'tier_and_details': {'0df078f33aa74a2e9696e0520c1a828a': {'tier': 'Bronze', 'id': '0df078f33aa74a2e9696e0520c1a828a', 'active': True, 'benefits': ['sports tickets']}, '699456451cc24f028d2aa99d7534c219': {'tier': 'Bronze', 'benefits': ['24 hour dedicated line', 'concierge services'], 'active': True, 'id': '699456451cc24f028d2aa99d7534c219'}}}
{'_id': ObjectId('5ca4bbcea2dd94ee58162a6b'), 'username': 'serranobrian', 'name': 'Leslie Martinez', 'address': 'Unit 2676 Box 9352\nDPO AA 38560', 'birthdate': datetime.datetime(1974, 11, 26, 14, 30, 20), 'email': 'tcrawford@gmail.com', 'accounts': [170945, 951849], 'tier_and_details': {'a15baf69a759423297f11ce6c7b0bc9a': {'tier': 'Pla

The code here returns the first 5 people who are younger than me. Note we first create a "date_cutoff" using _datetime.datetime_ (see [here](https://docs.python.org/3/library/datetime.html#examples-of-usage-datetime) for more details). After this we use the code "\{"\\$lt": date_cutoff\}" to apply the filter (where "\\$lt" represents "less than"; we could use "\\$gt" to achieve the opposite result). You can see the full list of operators [here](https://docs.mongodb.com/manual/reference/operator/query/).

We can also filter based on nested elements in the dictionary:

In [7]:
count = 0

for doc in collection.find():
    if count < 5:
        if len(doc['accounts']) > 3:
            print(doc)
            count += 1
    else:
        break

{'_id': ObjectId('5ca4bbcea2dd94ee58162a68'), 'username': 'fmiller', 'name': 'Elizabeth Ray', 'address': '9286 Bethany Glens\nVasqueztown, CO 22939', 'birthdate': datetime.datetime(1977, 3, 2, 2, 20, 31), 'email': 'arroyocolton@gmail.com', 'active': True, 'accounts': [371138, 324287, 276528, 332179, 422649, 387979], 'tier_and_details': {'0df078f33aa74a2e9696e0520c1a828a': {'tier': 'Bronze', 'id': '0df078f33aa74a2e9696e0520c1a828a', 'active': True, 'benefits': ['sports tickets']}, '699456451cc24f028d2aa99d7534c219': {'tier': 'Bronze', 'benefits': ['24 hour dedicated line', 'concierge services'], 'active': True, 'id': '699456451cc24f028d2aa99d7534c219'}}}
{'_id': ObjectId('5ca4bbcea2dd94ee58162a6a'), 'username': 'hillrachel', 'name': 'Katherine David', 'address': '55711 Janet Plaza Apt. 865\nChristinachester, CT 62716', 'birthdate': datetime.datetime(1988, 6, 20, 22, 15, 34), 'email': 'timothy78@hotmail.com', 'accounts': [462501, 228290, 968786, 515844, 377292], 'tier_and_details': {}}
{

We may also want to use logical operators (e.g. "AND", "OR", "NOT", etc.). Again we use the \\$ operator to do this:

In [8]:
query = {
    '$and':
        [
            {"birthdate": {"$lt": date_cutoff}}, 
            {'username': 'fmiller'}
        ]
}

for doc in collection.find(query):
    print(doc)

{'_id': ObjectId('5ca4bbcea2dd94ee58162a68'), 'username': 'fmiller', 'name': 'Elizabeth Ray', 'address': '9286 Bethany Glens\nVasqueztown, CO 22939', 'birthdate': datetime.datetime(1977, 3, 2, 2, 20, 31), 'email': 'arroyocolton@gmail.com', 'active': True, 'accounts': [371138, 324287, 276528, 332179, 422649, 387979], 'tier_and_details': {'0df078f33aa74a2e9696e0520c1a828a': {'tier': 'Bronze', 'id': '0df078f33aa74a2e9696e0520c1a828a', 'active': True, 'benefits': ['sports tickets']}, '699456451cc24f028d2aa99d7534c219': {'tier': 'Bronze', 'benefits': ['24 hour dedicated line', 'concierge services'], 'active': True, 'id': '699456451cc24f028d2aa99d7534c219'}}}


These queries can be a little clunky but the idea is that it is a dictionary where the logical operator is the key, and the conditions are a list inside this.

We can also get MongoDB to provide a sort order for our results (the SQL equivalent would be ORDER BY):

In [9]:
count = 0

for doc in collection.find({"birthdate": {"$lt": date_cutoff}}).sort("birthdate"):
    if count < 5:
        print(doc)
        count += 1
    else:
        break

{'_id': ObjectId('5ca4bbcea2dd94ee58162c23'), 'username': 'amanda70', 'name': 'Christopher Watson', 'address': '98287 Ryan Unions Suite 404\nPort Robertside, MD 88317', 'birthdate': datetime.datetime(1966, 7, 29, 17, 22, 6), 'email': 'srobinson@yahoo.com', 'accounts': [765127, 460206, 344075, 313926, 322023], 'tier_and_details': {'5cb805661d8f4fad9290fbd0976bb95a': {'tier': 'Silver', 'benefits': ['dedicated account representative'], 'active': True, 'id': '5cb805661d8f4fad9290fbd0976bb95a'}, '545c5449ab1a4df0953c60d6ecc38fd5': {'tier': 'Silver', 'benefits': ['concert tickets', 'dedicated account representative'], 'active': True, 'id': '545c5449ab1a4df0953c60d6ecc38fd5'}}}
{'_id': ObjectId('5ca4bbcea2dd94ee58162b92'), 'username': 'lisaroberts', 'name': 'Daniel Turner', 'address': '5498 Sandra Drive Suite 397\nWest Nancy, AZ 71732', 'birthdate': datetime.datetime(1966, 7, 30, 1, 15, 43), 'email': 'gmccarthy@yahoo.com', 'accounts': [576646], 'tier_and_details': {}}
{'_id': ObjectId('5ca4bb

If we wish to search in descending order we need to add a "-1" to the _sort()_ command:

In [10]:
count = 0

for doc in collection.find({"birthdate": {"$lt": date_cutoff}}).sort("birthdate", -1):
    if count < 5:
        print(doc)
        count += 1
    else:
        break

{'_id': ObjectId('5ca4bbcea2dd94ee58162b79'), 'username': 'peter77', 'name': 'Jacqueline Dixon', 'address': '0960 Evans Mountain Apt. 236\nSouth Daniel, NC 56886', 'birthdate': datetime.datetime(1981, 4, 21, 12, 21, 33), 'email': 'cindyarmstrong@gmail.com', 'accounts': [514695, 587785, 572301, 198855, 505260], 'tier_and_details': {'f2e43dbe067a4a3db9132ecd41e4e188': {'tier': 'Silver', 'benefits': ['concierge services'], 'active': True, 'id': 'f2e43dbe067a4a3db9132ecd41e4e188'}, '7477f5bf4451493ea12bbc39115d83e6': {'tier': 'Platinum', 'benefits': ['concierge services', 'dedicated account representative'], 'active': True, 'id': '7477f5bf4451493ea12bbc39115d83e6'}}}
{'_id': ObjectId('5ca4bbcea2dd94ee58162b1b'), 'username': 'cody34', 'name': 'Julie Erickson DDS', 'address': '4907 Morris Garden Apt. 647\nEmilystad, TX 73479', 'birthdate': datetime.datetime(1981, 4, 17, 17, 52, 49), 'email': 'sgreene@gmail.com', 'accounts': [103536, 971879], 'tier_and_details': {'86ee9babe3b14df28efd8027233d

## Joining Data

Unlike relational databases, joining collections/tables is not implicit. There is no declared schema (this is NoSQL after all) so no defined relationship between collections like there would be for tables in the relational model. However, this does not mean it is impossible! We just need to inject the schema on _read_ rather than _write._

The "best" way to do this is probably via ["aggregation pipelines"](https://docs.mongodb.com/manual/aggregation/), but these are fairly complicated so more than we will bother with here. Instead we can use more "Pythonic" solutions:

In [11]:
output = {}
first_query = []

for doc in collection.find({"birthdate": {"$lt": date_cutoff}}):
    try:
        # ignore any customer with no accounts or no username
        if doc['accounts'] and doc["username"]:  
            for acct in doc['accounts']:
                output[acct] = {"user": doc["username"], "limit": "Unknown"}
                first_query.append(acct)
    except:
        continue
    
second_collection = db.accounts

for doc in second_collection.find({"account_id": {"$in": first_query}}):
    output[doc["account_id"]]["limit"] = doc["limit"]
        
output

{371138: {'user': 'fmiller', 'limit': 9000},
 324287: {'user': 'fmiller', 'limit': 10000},
 276528: {'user': 'fmiller', 'limit': 10000},
 332179: {'user': 'fmiller', 'limit': 10000},
 422649: {'user': 'fmiller', 'limit': 10000},
 387979: {'user': 'fmiller', 'limit': 10000},
 170945: {'user': 'serranobrian', 'limit': 10000},
 951849: {'user': 'serranobrian', 'limit': 10000},
 721914: {'user': 'charleshudson', 'limit': 10000},
 817222: {'user': 'charleshudson', 'limit': 10000},
 973067: {'user': 'charleshudson', 'limit': 10000},
 260799: {'user': 'charleshudson', 'limit': 10000},
 87389: {'user': 'charleshudson', 'limit': 10000},
 627629: {'user': 'hmyers', 'limit': 10000},
 55958: {'user': 'hmyers', 'limit': 10000},
 771641: {'user': 'hmyers', 'limit': 10000},
 344885: {'user': 'glopez', 'limit': 10000},
 839927: {'user': 'glopez', 'limit': 10000},
 853542: {'user': 'glopez', 'limit': 10000},
 987709: {'user': 'wesley20', 'limit': 10000},
 571880: {'user': 'patricia44', 'limit': 10000},

This may look a little intimidating but actually its nothing massively new. We create dictionary to post our joined results to. Secondly we create an empty list in which we will add every account ID associated with a customer older than our cut-off.

The first for loop executes the above - adding anyone older than our date who has a username and at least one account to the dictionary. The key for our disctionary is the account ID (so each customer can be listed multiple times) and the values are a nested dictionary containing the "username" and account "limit" (we set the initial "limit" to be "Unknown" as we need to get this from the other collection). We also keep a list of all the account IDs ("first_query").

After this we loop through the _accounts_ collection but only extract records where the "account_id" is included in the "first_query" list (in other words a LEFT JOIN in SQL terminology). With these filtered results we update the dictionary to change the limit to whatever the limit is in the _accounts_ collection.

We could imagine this as a SQL query as something like the following (note - this is SQL code so won't run in the Notebook):

In [12]:
'''
SELECT account, username AS user
FROM customers
WHERE birthdate < DATE("1981/06/13")
LEFT JOIN accounts
ON customers.account = accounts.account_ID;
'''

'\nSELECT account, username AS user\nFROM customers\nWHERE birthdate < DATE("1981/06/13")\nLEFT JOIN accounts\nON customers.account = accounts.account_ID;\n'

_Note however this SQL would not work as we do not have one record per account ID but each record has a list of account IDs. We would typically have designed around this with an account_ID to customer_ID lookup table. However, this does highlight the extra flexibility of NoSQL - that we can have a list within a field within a collection._

Our final step will be outputing our joined table, using _pandas:_

In [13]:
import pandas as pd

outputdf = pd.DataFrame(output)
outputdf = outputdf.transpose()
outputdf.reset_index(inplace=True)
outputdf.rename(columns = {"index": "account_id"}, inplace=True)

print(outputdf.head())

outputdf.to_csv("mongoout.csv", index=False)

   account_id     user  limit
0      371138  fmiller   9000
1      324287  fmiller  10000
2      276528  fmiller  10000
3      332179  fmiller  10000
4      422649  fmiller  10000


Slightly tougher Notebook - well done if you made it this far!!!